In [1]:
import h5py,pandas as pd, numpy as np
import os.path
import sys
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale, StandardScaler, MinMaxScaler
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTEENN, SMOTETomek
from tensorflow import keras
from tensorflow.keras import layers
import itertools
import matplotlib.pyplot as plt

Aquisition des données pour utilisation dans Jupyter (local)

In [2]:
from data import dataread, datatreat_A1
from models import cnn_1, cnn_2, cnn_3, cnn_4, cnn_5, cnn_6, cnn_7
from test import test_1
from tools import save_model, save_results, plot_loss_acc_history

In [3]:
X, y, X_final = dataread()

In [ ]:
X_train, X_test, y_train, y_test, prop_HF = datatreat_A1(X, y, train_size=0.8, Shuffle=True, preprocess="None", ratio="50/50", balancing_method="rem/SMOTE")

In [4]:
epochs=150
batch_size=70
validation_split=0.1